In [6]:
import torch
from torch import nn # building blocks for neural networks
import matplotlib.pyplot as plt

torch.__version__

'1.12.1'

## 1. Prepare & Load Data

In [19]:
# Create data
X = torch.arange(0, 1, 0.02).unsqueeze(dim=1)
y = .7 * X + .3

# Split into train & test
split = int(.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

## 2. Build Model

In [31]:
# Create Linear Regression Model Class
class LinearRegressionNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1, 
                                               requires_grad=True, #Can this be updated via gradient descent?
                                               dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1, 
                                               requires_grad=True,
                                               dtype=torch.float))
        
    def forward(self, x):
        return self.weights * x + self.bias

In [47]:
torch.manual_seed(42)
model = LinearRegressionNN()

In [54]:
list(model.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [61]:
model.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [51]:
model.bias

Parameter containing:
tensor([0.1288], requires_grad=True)

### Make predictions

In [63]:
with torch.inference_mode():
    y_preds = model(X_test)

In [64]:
y_preds

tensor([[0.3982],
        [0.4049],
        [0.4116],
        [0.4184],
        [0.4251],
        [0.4318],
        [0.4386],
        [0.4453],
        [0.4520],
        [0.4588]])